In [30]:
!pip install unidecode
!pip install transformers==4.2.2
!pip install pyreadr
!pip install unidecode
!pip install ftfy
!pip install git+https://github.com/legalnlp21/legalnlp

  Cloning https://github.com/legalnlp21/legalnlp to /tmp/pip-req-build-kqwmck1c
  Running command git clone -q https://github.com/legalnlp21/legalnlp /tmp/pip-req-build-kqwmck1c


In [31]:
from IPython.display import Image
from IPython.display import clear_output
import pickle
import random
import unidecode
import re
import ftfy
import os
import copy
import torch
import torch.nn as nn
import torch.utils.data as tdata
import torch.optim as optim
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import BertForPreTraining, BertModel, BertTokenizer, BertForMaskedLM, BertForNextSentencePrediction, BertForQuestionAnswering
# manipulação numérica e de dataframes
import numpy as np
import pandas as pd
# gráficos e ajustes visuais
import matplotlib.pyplot as plt
import textwrap
from tqdm import tqdm

In [33]:
from legalnlp.get_premodel import *
from legalnlp.clean_functions import *

In [ ]:
get_premodel('bert')

In [34]:
data=pd.read_csv('https://raw.githubusercontent.com/legalnlp21/legalnlp/main/demo/data_base.csv')
data.drop(columns=['Unnamed: 0'],inplace=True)
data['text'] = data['text'].apply(lambda x:clean_bert(x))
model = '/content/BERTikal (1)/BERTikal'
tokenizer= '/content/BERTikal (1)/BERTikal/vocab.txt'
data

,text,label
0,Ofício Requisitório - Comunicação - Protocolo ...,H:Arquivado
1,Suspensão do Prazo Prazo referente ao usuário ...,H:Ativo
2,Certidão de Publicação Expedida Relação :0274/...,H:Ativo
3,Embargos de Declaração Juntados Nº Protocolo: ...,H:Ativo
4,Certidão de Trânsito em Julgado com Baixa Expe...,H:Arquivado
...,...,...
5088,Petição Juntada Juntada a petição diversa - Ti...,H:Ativo
5089,Suspensão do Prazo Prazo referente ao usuário ...,H:Ativo
5090,Tipo do Movimento:Arquivamento Tipo de arquiva...,H:Arquivado
5091,Ofício Juntado Juntada a petição diversa - Tip...,H:Ativo


In [ ]:
def extract_features_bert(path_model, path_tokenizer, data,gpu=True):
    if gpu:
        device = torch.device('cuda')
        os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
        bert_tokenizer = BertTokenizer.from_pretrained(path_tokenizer, do_lower_case=False)
        bert_model = BertModel.from_pretrained(path_model).to(device)
        bert = data.apply(lambda x: bert_tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))
        data_text = list(data)
        encoded_inputs = bert_tokenizer(data_text, padding=True, truncation=True, max_length=512, return_tensors="pt")
        input_ids = encoded_inputs['input_ids'].to(device)
        clear_output()
        # Criando o nosso vetor de features
        features = []
    
        # Aplicando o modelo pré-treinado em cada frase e adicionando-o ao nosso vetor
    
        for i in tqdm(range(len(data)), mininterval=40, maxinterval=500):
    
            with torch.no_grad():
    
                last_hidden_states = bert_model(
                    input_ids[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()
            features.append(last_hidden_states)
    else:
        bert_tokenizer = BertTokenizer.from_pretrained(path_tokenizer, do_lower_case=False)
        bert_model = BertModel.from_pretrained(path_model)
        bert = data.apply(lambda x: bert_tokenizer.encode(
            x, add_special_tokens=True, max_length=512, truncation=True))
        encoded_inputs = bert_tokenizer(
            data_text, padding=True, truncation=True, max_length=512, return_tensors="pt")
        input_ids = encoded_inputs['input_ids']
        # Criando o nosso vetor de features
        features = []
    
        # Aplicando o modelo pré-treinado em cada frase e adicionando-o ao nosso vetor
    
        for i in tqdm(range(len(data))):
    
            with torch.no_grad():
    
                last_hidden_states = bert_model(
                    input_ids[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()
            features.append(last_hidden_states)
    features = np.array(features)
    features[:2]
    df_features = pd.DataFrame(features)
    return df_features



    # os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    #device = torch.device('cuda:0')
features= extract_features_bert(model,tokenizer,data['text'])

 96%|█████████▌| 4884/5093 [03:20<00:08, 24.35it/s]

In [ ]:
features